# Simple Recommender System - Product Recommender

## 1. Importing Libraries

In [3]:
#import library to be used in the project
import numpy as np
import pandas as pd
import html
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## 2. File Reading and Features Engineering: products

In [4]:
# Read the electronic products file
products_dataset_path = r'C:\Users\User\Desktop\product-recommender-system\dataset\subset_meta_Electronics.json'
products = pd.read_json(products_dataset_path, lines=True)

# Output the first 10 rows
products.head(10)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Electronics, Camera &amp; Photo, Video Survei...",,[The following camera brands and models have b...,,Genuine Geovision 1 Channel 3rd Party NVR IP S...,[],,GeoVision,"[Genuine Geovision 1 Channel NVR IP Software, ...","[>#3,092 in Tools &amp; Home Improvement &gt; ...",[],Camera &amp; Photo,,"January 28, 2014",$65.00,0011300000,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Electronics, Camera &amp; Photo]",,[This second edition of the Handbook of Astron...,,"Books ""Handbook of Astronomical Image Processi...",[0999470906],,33 Books Co.,[Detailed chapters cover these fundamental top...,"[>#55,933 in Camera &amp; Photo (See Top 100 i...","[0943396670, 1138055360, 0999470906]",Camera &amp; Photo,,"June 17, 2003",,0043396828,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Electronics, eBook Readers &amp; Accessories,...",,[A zesty tale. (Publishers Weekly)<br /><br />...,,One Hot Summer,"[0425167798, 039914157X]",,Visit Amazon's Carolina Garcia Aguilera Page,[],"3,105,177 in Books (",[],Books,,,$11.49,0060009810,[],[],NaN
3,"[Electronics, eBook Readers & Accessories, eBo...",,[],,Hurray for Hattie Rabbit: Story and pictures (...,"[0060219521, 0060219580, 0060219394]",,Visit Amazon's Dick Gackenbach Page,[],"2,024,298 in Books (","[0060219521, 0060219475, 0060219394]",Books,,,.a-section.a-spacing-mini{margin-bottom:6px!im...,0060219602,[],[],NaN
4,"[Electronics, eBook Readers & Accessories, eBo...",,[&#8220;sex.lies.murder.fame. is brillllli&#82...,,sex.lies.murder.fame.: A Novel,[],,Visit Amazon's Lolita Files Page,[],"3,778,828 in Books (",[],Books,,,$13.95,0060786817,[],[],NaN
5,"[Electronics, eBook Readers &amp; Accessories,...",,"[, ]",,College Physics,"[0073049557, 0134454170, 1118142063, 007733968...",,Visit Amazon's Alan Giambattista Page,[],"3,330,771 in Books (","[0073512141, 0077339681, 0073049557, 007304956...",Books,,,,0070524076,[],[],NaN
6,"[Electronics, eBook Readers & Accessories, eBo...",,[GIRL WITH A ONE-TRACK MIND: CONFESSIONS OF TH...,,Girl with a One-track Mind: Confessions of the...,[0330509691],,ABBY LEE,[],"3,304,037 in Books (",[B0719LDQR1],Books,,,$4.76,0091912407,[],[],NaN
7,"[Electronics, Portable Audio & Video, MP3 & MP...",,[Support system: Windows XP/Vsita/7 * SNR: 85d...,,abcGoodefg&reg; 4GB USB 2.0 Mp3 Music Player w...,"[B01NAJ3KQB, B00WYSPT0C, B00AF40U5G, B00OFVNM4...",,Crazy Cart,[Package Content: 1 x Display MP3 Player 1 x E...,"[>#177,454 in Electronics (See Top 100 in Elec...","[B01NAJ3KQB, B00OFVNM4G, B00L41WY8K, B07F34PNP...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","December 28, 2012",,0101635370,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
8,"[Electronics, Headphones, Earbud Headphones]",,"[, <b>True High Definition Sound:</b><br>With ...",,Wireless Bluetooth Headphones Earbuds with Mic...,[],,Enter The Arena,[Superb Sound Quality: Plays crystal clear aud...,[>#950 in Cell Phones & Accessories (See Top 1...,[],Home Audio & Theater,,"October 23, 2017",$7.99,0132492776,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
9,"[Electronics, Computers &amp; Accessories, Com...",,[],,Kelby Training DVD: Mastering Blend Modes in A...,[],,Kelby Training,[],"[>#932,732 in Computers &amp; Accessories &gt;...",[],Computers,,"December 9, 2011",,0132793040,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [5]:
print(products.shape)
# output: (rows, columns)

(104802, 19)


In [6]:
# retrieving column name
products.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

### 2.1 General Cleansing

#### 2.1.1 Drop Unnecessary columns

In [7]:
# keep: asin, title, brand, main_cat, and price (some columns are for filtering usage)
products = products.drop(columns=['category', 'tech1', 'description', 'fit', 'also_buy', 'tech2',
       'feature', 'rank', 'also_view', 'similar_item', 'date', 'imageURL', 'imageURLHighRes', 'details'], errors='ignore')

products.columns

Index(['title', 'brand', 'main_cat', 'price', 'asin'], dtype='object')

In [8]:
products = products[['asin', 'title', 'brand', 'price', 'main_cat']]
products.columns = ['product_id', 'product_name', 'brand_or_author', 'price', 'main_category']

In [9]:
products.head(10)

,product_id,product_name,brand_or_author,price,main_category
0,0011300000,Genuine Geovision 1 Channel 3rd Party NVR IP S...,GeoVision,$65.00,Camera &amp; Photo
1,0043396828,"Books ""Handbook of Astronomical Image Processi...",33 Books Co.,,Camera &amp; Photo
2,0060009810,One Hot Summer,Visit Amazon's Carolina Garcia Aguilera Page,$11.49,Books
3,0060219602,Hurray for Hattie Rabbit: Story and pictures (...,Visit Amazon's Dick Gackenbach Page,.a-section.a-spacing-mini{margin-bottom:6px!im...,Books
4,0060786817,sex.lies.murder.fame.: A Novel,Visit Amazon's Lolita Files Page,$13.95,Books
5,0070524076,College Physics,Visit Amazon's Alan Giambattista Page,,Books
6,0091912407,Girl with a One-track Mind: Confessions of the...,ABBY LEE,$4.76,Books
7,0101635370,abcGoodefg&reg; 4GB USB 2.0 Mp3 Music Player w...,Crazy Cart,,All Electronics
8,0132492776,Wireless Bluetooth Headphones Earbuds with Mic...,Enter The Arena,$7.99,Home Audio & Theater
9,0132793040,Kelby Training DVD: Mastering Blend Modes in A...,Kelby Training,,Computers


#### 2.1.2 Remove Duplicates

In [10]:
products.shape

(104802, 5)

In [11]:
products.duplicated().sum()

30368

In [12]:
products = products.drop_duplicates()

products.shape

(74434, 5)

#### 2.1.3 Format Strings

In [13]:
# Defining text cleaning function

def text_cleaning(text):
    # 1. convert any HTML entities in the text to their corresponding characters
    # e.g. &amp; to &, &quot; to ", &reg; to ®
    text = html.unescape(text)
    
    # 2. convert to lower case
    return text.lower()

text = "&amp; &quot &reg;"
cleaned_text = text_cleaning(text)
print(cleaned_text)

& " ®


In [14]:
# List of columns to be cleaned
cols_to_clean = ['product_id', 'product_name', 'brand_or_author', 'price', 'main_category']

# Apply the text cleaning function to each column
for col in cols_to_clean:
    products[col] = products[col].apply(lambda x: text_cleaning(x))

In [15]:
products.head(10)

,product_id,product_name,brand_or_author,price,main_category
0,0011300000,genuine geovision 1 channel 3rd party nvr ip s...,geovision,$65.00,camera & photo
1,0043396828,"books ""handbook of astronomical image processi...",33 books co.,,camera & photo
2,0060009810,one hot summer,visit amazon's carolina garcia aguilera page,$11.49,books
3,0060219602,hurray for hattie rabbit: story and pictures (...,visit amazon's dick gackenbach page,.a-section.a-spacing-mini{margin-bottom:6px!im...,books
4,0060786817,sex.lies.murder.fame.: a novel,visit amazon's lolita files page,$13.95,books
5,0070524076,college physics,visit amazon's alan giambattista page,,books
6,0091912407,girl with a one-track mind: confessions of the...,abby lee,$4.76,books
7,0101635370,abcgoodefg® 4gb usb 2.0 mp3 music player with ...,crazy cart,,all electronics
8,0132492776,wireless bluetooth headphones earbuds with mic...,enter the arena,$7.99,home audio & theater
9,0132793040,kelby training dvd: mastering blend modes in a...,kelby training,,computers


### 2.2 Cleaning 'main_category' column

In [16]:
main_category_df = products.groupby('main_category').size().reset_index(name='count')
main_category_df

,main_category,count
0,"<img src=""https://images-na.ssl-images-amazon....",38
1,"<img src=""https://images-na.ssl-images-amazon....",129
2,"<img src=""https://m.media-amazon.com/images/g/...",1
3,"<img src=""https://m.media-amazon.com/images/g/...",1
4,all beauty,30
5,all electronics,23269
6,amazon devices,37
7,amazon home,428
8,appliances,2
9,"arts, crafts & sewing",110


In [17]:
# The four image HTML elements are actually belongs to 'amazon fashion'
for i in (main_category_df.loc[0:3, 'main_category']):
    print(i)

# There are total 169 products for amazon fashion
total_af = {'main_category': 'Total', 'count': len(products.loc[products['main_category'].str.contains('amazon fashion')])}
main_category_df.loc[main_category_df['main_category'].str.contains('amazon fashion')].append(total_af, ignore_index=True)

<img src="https://images-na.ssl-images-amazon.com/images/g/01/nav2/images/gui/amazon-fashion-store-new._cb520838675_.png" class="nav-categ-image" alt="amazon fashion" />
<img src="https://images-na.ssl-images-amazon.com/images/g/01/nav2/images/gui/amazon-fashion-store-new._cb520838675_.png" class="nav-categ-image" alt="amazon fashion"/>
<img src="https://m.media-amazon.com/images/g/01/nav2/images/gui/amazon-fashion-store-new._cb520838675_.png" class="nav-categ-image" alt="amazon fashion" />
<img src="https://m.media-amazon.com/images/g/01/nav2/images/gui/amazon-fashion-store-new._cb520838675_.png" class="nav-categ-image" alt="amazon fashion"/>


,main_category,count
0,"<img src=""https://images-na.ssl-images-amazon....",38
1,"<img src=""https://images-na.ssl-images-amazon....",129
2,"<img src=""https://m.media-amazon.com/images/g/...",1
3,"<img src=""https://m.media-amazon.com/images/g/...",1
4,Total,169


In [18]:
# assign the four image HTML elements to 'amazon fashion' in main_category_df data frame
main_category_df.loc[main_category_df['main_category'].str.contains('amazon fashion'), 'main_category'] = 'amazon fashion'

main_category_df.groupby('main_category').sum()

,count
main_category,
all beauty,30
all electronics,23269
amazon devices,37
amazon fashion,169
amazon home,428
appliances,2
"arts, crafts & sewing",110
automotive,417
baby,19


In [19]:
# assign the four image HTML elements to 'amazon fashion' in products data frame
products.loc[products['main_category'].str.contains('amazon fashion'), 'main_category'] = 'amazon fashion'

products.groupby('main_category').size().reset_index(name='count')

,main_category,count
0,all beauty,30
1,all electronics,23269
2,amazon devices,37
3,amazon fashion,169
4,amazon home,428
5,appliances,2
6,"arts, crafts & sewing",110
7,automotive,417
8,baby,19
9,books,335


### 2.3 Cleaning 'brand_or_author' column

In [20]:
# The visit amazon's .... page pattern is actually the author.
pattern = "^visit amazon's (.*) page$"
mask = products['brand_or_author'].str.contains(pattern)
selected_columns = ['brand_or_author', 'main_category']
visit_amazon_pattern = products.loc[mask, selected_columns]

visit_amazon_pattern

,brand_or_author,main_category
2,visit amazon's carolina garcia aguilera page,books
3,visit amazon's dick gackenbach page,books
4,visit amazon's lolita files page,books
5,visit amazon's alan giambattista page,books
10,visit amazon's claire messud page,books
...,...,...
1403,visit amazon's dan wells page,books
1405,visit amazon's ismael cala page,books
1439,visit amazon's maría nuñez quesada page,books
68972,visit amazon's karin slaughter page,books


In [21]:
visit_amazon_pattern.groupby('main_category').size().reset_index(name='count')

,main_category,count
0,books,181


In [22]:
# Clean the brand or author column
def clean_author_string(author_string):
    author_string = author_string.replace("visit amazon's ", "")
    author_string = author_string.replace(" page", "")
    return author_string

products['brand_or_author'] = products['brand_or_author'].apply(clean_author_string)

In [23]:
# result
products.head(10)

,product_id,product_name,brand_or_author,price,main_category
0,0011300000,genuine geovision 1 channel 3rd party nvr ip s...,geovision,$65.00,camera & photo
1,0043396828,"books ""handbook of astronomical image processi...",33 books co.,,camera & photo
2,0060009810,one hot summer,carolina garcia aguilera,$11.49,books
3,0060219602,hurray for hattie rabbit: story and pictures (...,dick gackenbach,.a-section.a-spacing-mini{margin-bottom:6px!im...,books
4,0060786817,sex.lies.murder.fame.: a novel,lolita files,$13.95,books
5,0070524076,college physics,alan giambattista,,books
6,0091912407,girl with a one-track mind: confessions of the...,abby lee,$4.76,books
7,0101635370,abcgoodefg® 4gb usb 2.0 mp3 music player with ...,crazy cart,,all electronics
8,0132492776,wireless bluetooth headphones earbuds with mic...,enter the arena,$7.99,home audio & theater
9,0132793040,kelby training dvd: mastering blend modes in a...,kelby training,,computers


### 2.4 Cleaning 'price' column

In [24]:
# Clean the 'price' column
def clean_price_string (price_string):
    if price_string.startswith('$'):
        price_string = price_string.replace('$', '')
    else:
        price_string = '0'
    try:
        price_float = float(price_string)
    except ValueError:
        price_float = 0.0
    return price_float

products['price'] = products['price'].apply(clean_price_string)

In [25]:
# result
products.head(10)

,product_id,product_name,brand_or_author,price,main_category
0,0011300000,genuine geovision 1 channel 3rd party nvr ip s...,geovision,65.00,camera & photo
1,0043396828,"books ""handbook of astronomical image processi...",33 books co.,0.00,camera & photo
2,0060009810,one hot summer,carolina garcia aguilera,11.49,books
3,0060219602,hurray for hattie rabbit: story and pictures (...,dick gackenbach,0.00,books
4,0060786817,sex.lies.murder.fame.: a novel,lolita files,13.95,books
5,0070524076,college physics,alan giambattista,0.00,books
6,0091912407,girl with a one-track mind: confessions of the...,abby lee,4.76,books
7,0101635370,abcgoodefg® 4gb usb 2.0 mp3 music player with ...,crazy cart,0.00,all electronics
8,0132492776,wireless bluetooth headphones earbuds with mic...,enter the arena,7.99,home audio & theater
9,0132793040,kelby training dvd: mastering blend modes in a...,kelby training,0.00,computers


## 3. File Reading and Features Engineering: ratings

In [26]:
# Read the ratings file
ratings_dataset_path = r'C:\Users\User\Desktop\product-recommender-system\dataset\electronic_user.csv'
ratings = pd.read_csv(ratings_dataset_path, names=['user_id', 'product_id','rating','timestamp'], index_col=False)

# Output the first 10 rows
ratings.head(10)

,user_id,product_id,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
5,A1QGNMC6O1VW39,0511189877,5.0,1397433600
6,A3J3BRHTDRFJ2G,0511189877,2.0,1397433600
7,A2TY0BTJOTENPG,0511189877,5.0,1395878400
8,A34ATBPOK6HCHY,0511189877,5.0,1395532800
9,A89DO69P0XZ27,0511189877,5.0,1395446400


In [27]:
ratings.columns

Index(['user_id', 'product_id', 'rating', 'timestamp'], dtype='object')

### 3.1 Features Engineering

#### 3.1.1 Check Duplicates ( no duplicated rows )

In [28]:
ratings.shape

(7824482, 4)

In [29]:
# must check duplicates first before dropping the columns
# because all the four columns, especially the timestamp, are determining the duplication

ratings.duplicated().sum()

# no need to drop duplicates
# no need: ratings = ratings.drop_duplicates()

0

#### 3.1.2 Drop Unnecessary Columns

In [30]:
# keep: product_id and rating  (each rating is valuable only)
# drop: user_id and timestamp
ratings = ratings.drop(columns=['user_id', 'timestamp'], errors='ignore')

ratings.columns

Index(['product_id', 'rating'], dtype='object')

In [31]:
ratings.head(10)

,product_id,rating
0,0132793040,5.0
1,0321732944,5.0
2,0439886341,1.0
3,0439886341,3.0
4,0439886341,1.0
5,0511189877,5.0
6,0511189877,2.0
7,0511189877,5.0
8,0511189877,5.0
9,0511189877,5.0


#### 3.1.3 Create  'rating_average' and 'rating_count' from 'rating'

In [32]:
# Calculate the average rating and count of ratings for each product_id
ratings = ratings.groupby('product_id').agg(rating_average=('rating', 'mean'), rating_count=('rating', 'count'))

# use (0,1,2,3,4,5,6 as row indication instead of product_id)
ratings = ratings.reset_index()

ratings.head(10)

,product_id,rating_average,rating_count
0,0132793040,5.000000,1
1,0321732944,5.000000,1
2,0439886341,1.666667,3
3,0511189877,4.500000,6
4,0528881469,2.851852,27
5,0558835155,3.000000,1
6,059400232X,5.000000,3
7,0594012015,2.000000,8
8,0594017343,1.000000,1
9,0594017580,3.000000,1


In [33]:
# Top 10 products with the highest rating_count first then highest rating_average
ratings.sort_values(by=['rating_count','rating_average'], ascending=[False, False]).head(10)

,product_id,rating_average,rating_count
308398,B0074BW614,4.491504,18244
429572,B00DR0PDNE,3.931020,16454
327308,B007WTAJTO,4.424005,14172
102804,B0019EHU8G,4.754497,12285
296625,B006GWO5WK,4.314657,12226
178601,B003ELYQGG,4.392528,11617
178813,B003ES5ZUU,4.704749,10276
323013,B007R5YDYA,4.690926,9907
289775,B00622AG6S,4.420136,9823
30276,B0002L5R78,4.448614,9487


## 4. Merge 'products' and 'ratings' into 'products_merge'

In [45]:
# Merge the products and ratings dataframes (keep all the products records)
products_merge = products.merge(ratings, on='product_id', how='left')

# Output the first 10 rows
products_merge.head(10)

,product_id,product_name,brand_or_author,price,main_category,rating_average,rating_count
0,0011300000,genuine geovision 1 channel 3rd party nvr ip s...,geovision,65.00,camera & photo,NaN,NaN
1,0043396828,"books ""handbook of astronomical image processi...",33 books co.,0.00,camera & photo,NaN,NaN
2,0060009810,one hot summer,carolina garcia aguilera,11.49,books,NaN,NaN
3,0060219602,hurray for hattie rabbit: story and pictures (...,dick gackenbach,0.00,books,NaN,NaN
4,0060786817,sex.lies.murder.fame.: a novel,lolita files,13.95,books,NaN,NaN
5,0070524076,college physics,alan giambattista,0.00,books,NaN,NaN
6,0091912407,girl with a one-track mind: confessions of the...,abby lee,4.76,books,NaN,NaN
7,0101635370,abcgoodefg® 4gb usb 2.0 mp3 music player with ...,crazy cart,0.00,all electronics,NaN,NaN
8,0132492776,wireless bluetooth headphones earbuds with mic...,enter the arena,7.99,home audio & theater,NaN,NaN
9,0132793040,kelby training dvd: mastering blend modes in a...,kelby training,0.00,computers,5.0,1.0


In [46]:
products_merge.shape

(74434, 7)

### 4.1 Replace NaN

In [47]:
products_merge.isnull().sum()

product_id             0
product_name           0
brand_or_author        0
price                  0
main_category          0
rating_average     73794
rating_count       73794
dtype: int64

In [48]:
products_merge[products_merge['main_category'] == 'all electronics'].count()

product_id         23269
product_name       23269
brand_or_author    23269
price              23269
main_category      23269
rating_average       134
rating_count         134
dtype: int64

In [49]:
products_merge[products_merge['main_category'] == 'all electronics'].isnull().sum()

# Too many null rating value of the dataset
# this could be a problem if we want to recommend more products
# But still tolerable if we just want to recommend top few products

product_id             0
product_name           0
brand_or_author        0
price                  0
main_category          0
rating_average     23135
rating_count       23135
dtype: int64

In [50]:
products_merge.fillna({'rating_average': 0.0, 'rating_count': 0}, inplace=True)

In [51]:
products_merge.isnull().sum()

product_id         0
product_name       0
brand_or_author    0
price              0
main_category      0
rating_average     0
rating_count       0
dtype: int64

## 5. Simple Recommender System

### 5.1 Simple Rating Sort

▪ Sorting 'products_merge' by multiple columns.

▪ Issue: Even if the **rating_count** for a product is very high, but it can have a lower **rating_average**.

In [53]:
# Top 10 products with the highest rating_count first then highest rating_average
products_merge.sort_values(by=['rating_count','rating_average'], ascending=[False, False]).head(50)

,product_id,product_name,brand_or_author,price,main_category,rating_average,rating_count
201,0972683275,"videosecu 24"" long arm tv wall mount low profi...",videosecu,34.99,all electronics,4.470980,1051.0
464,1400501466,"barnes & noble nook tablet 16gb (color, bntv250)",barnes & noble,0.00,computers,3.560000,250.0
461,1400599997,barnes & noble nook ebook reader (wifi + 3g)[b&w],barnes & noble,0.00,all electronics,3.490991,222.0
992,9573212919,external usb dvd/cd,epartsdom,14.86,computers,3.879397,199.0
1872,9983891212,premium canon mini hdmi cable 10 feet,lasus,0.00,all electronics,4.442529,174.0
471,1400532620,barnes and noble nook ebook reader (wifi only)...,barnes & noble,0.00,all electronics,3.684211,171.0
1003,9625993428,mycarryingcase black mini microphone for iphon...,mycarryingcase,2.08,portable audio & accessories,3.420168,119.0
946,8862935293,moleskine kindle cover with reporter-style not...,moleskine,0.00,amazon devices,3.915789,95.0
1945,9984984354,garmin nuvi 1450 gps standard red led wall / a...,garmin,9.95,gps & navigation,3.075949,79.0
1459,9888002198,universial travel ac wall adapter usb charger,mycarryingcase,7.24,all electronics,3.971429,70.0


### 5.2 Weighted Rating

▪ A *weighted rating* that takes into account the **rating_average** and the **rating_count** it has accumulated.

▪ We can calculate the Weighted Rating Score into a new 'score' column.

▪ The formula of weighted rating is as follows:

<img src="weighted_rating.png" width="600">

\>>> **v** is the number of rating for the product (represented by **rating_count**)

\>>> **m** is the **minimum rating count** required to be listed in the chart (to be calculated)

\>>> **R** is the average rating of the product (represented by **rating_average**)

\>>> **C** is the **mean of rating average** across the whole dataframe (to be calculated)

In [103]:
# describe() - can analyse the dataframe overallly
# C can be viewed using describe().

products_merge.describe()

# From the output:
# get C: mean of **rating_average** of product

,price,rating_average,rating_count
count,74434.000000,74434.000000,74434.000000
mean,8.630102,0.034153,0.061249
std,38.781412,0.385522,4.318074
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,3.760000,0.000000,0.000000
max,999.990000,5.000000,1051.000000


In [224]:
# C, the mean of rating average across the whole dataframe
C = products_merge['rating_average'].mean()
C

0.03415334719903093

In [225]:
# m, minimum rating count required to be listed in the chart

# consider the 99.5th percentile.

# for a product to be recommended, it must have more votes than at least 99.5% of the products.

m = products_merge['rating_count'].quantile(0.995) 
m

1.0

In [226]:
q_products = products_merge.copy().loc[products_merge['rating_count'] >= m]

q_products

,product_id,product_name,brand_or_author,price,main_category,rating_average,rating_count
9,0132793040,kelby training dvd: mastering blend modes in a...,kelby training,0.00,computers,5.000000,1.0
23,0321732944,kelby training dvd: adobe photoshop cs5 crash ...,kelby training,0.00,computers,5.000000,1.0
37,0511189877,clikr-5 time warner cable remote control ur5u-...,urc,0.00,all electronics,4.500000,6.0
40,0528881469,rand mcnally 528881469 7-inch intelliroute tnd...,rand mcnally,0.00,all electronics,2.851852,27.0
47,0558835155,polaroid pbm2200 pc / gaming stereo headphones...,polaroid,0.00,all electronics,3.000000,1.0
...,...,...,...,...,...,...,...
2271,9990950636,premium 6 foot high speed hdmi cable for your ...,a days tech,4.25,all electronics,4.285714,7.0
2272,9991330038,mobile action ma-8072p usb data cable software...,mobile excess,0.00,computers,1.000000,1.0
2274,9991546634,cellet microsd 4gb memory card for palm pre ph...,cellet,0.00,computers,1.000000,1.0
2278,9991549358,cellet microsd 4gb memory card for samsung sgh...,cellet,0.00,computers,5.000000,1.0


In [227]:
# Function that computes the weighted rating of each product
def weighted_rating(x, m = m, C = C):
    
    v = x['rating_count']
    R = x['rating_average']
    
    # Calculation based on the IMDB formula
    return (v / (v + m) * R) + (m / (m + v) * C)

In [228]:
q_products['score'] = q_products.apply(weighted_rating, axis=1)

q_products.head(10)

,product_id,product_name,brand_or_author,price,main_category,rating_average,rating_count,score
9,0132793040,kelby training dvd: mastering blend modes in a...,kelby training,0.00,computers,5.000000,1.0,2.517077
23,0321732944,kelby training dvd: adobe photoshop cs5 crash ...,kelby training,0.00,computers,5.000000,1.0,2.517077
37,0511189877,clikr-5 time warner cable remote control ur5u-...,urc,0.00,all electronics,4.500000,6.0,3.862022
40,0528881469,rand mcnally 528881469 7-inch intelliroute tnd...,rand mcnally,0.00,all electronics,2.851852,27.0,2.751220
47,0558835155,polaroid pbm2200 pc / gaming stereo headphones...,polaroid,0.00,all electronics,3.000000,1.0,1.517077
52,0594012015,barnes & noble nook power kit in carbon bnadpn31,barnes & noble,0.00,all electronics,2.000000,8.0,1.781573
53,0594017580,jonathan adler punctuation cover in black,barnes & nobles,8.99,portable audio & accessories,3.000000,1.0,1.517077
56,0594033896,nook color clear screen protector kit 2-pack,barnes & noble,1.96,portable audio & accessories,4.400000,5.0,3.672359
58,0594287995,kate spade rain or shine magazine cover for no...,barnes & noble,0.00,all electronics,5.000000,1.0,2.517077
59,0594017343,barnes noble nook simple touch reader brnv300+...,barnes and noble,0.00,cell phones & accessories,1.000000,1.0,0.517077


### 5.3 Result using Weighted Rating

#### 5.3.1 Recommending Top 20 Products

▪ Sort q_proucts in descending order based on the score feature column.

▪ Output the product_name, main_category, brand_or_author, price, rating_average, rating_count, and weighted rating (score) of the top 20 products.

In [230]:
# Sort products based on 'score' and recommend the top 20 products
top_20_proucts = q_products.sort_values('score', ascending = False).head(20).reset_index()

columns = ['product_name', 'main_category','brand_or_author', 'price',  'rating_average', 'rating_count', 'score']
top_20_proucts = top_20_proucts [columns]

top_20_proucts.index = top_20_proucts.index + 1

top_20_proucts

,product_name,main_category,brand_or_author,price,rating_average,rating_count,score
1,barnes noble 5010490303 lautner e-reader cover,portable audio & accessories,barnes & noble,22.49,4.809524,21.0,4.592462
2,professional kingston microsdhc 4gb card for g...,computers,kingston,0.00,4.645161,62.0,4.571971
3,"perfect pixel 30"" yamakasi 301 sparta 2560x160...",computers,yamakasi,0.00,4.724138,29.0,4.567805
4,"videosecu 24"" long arm tv wall mount low profi...",all electronics,videosecu,34.99,4.470980,1051.0,4.466763
5,premium canon mini hdmi cable 10 feet,all electronics,lasus,0.00,4.442529,174.0,4.417338
6,bargain cable db9 female / db9 female null mod...,computers,stsi,5.05,4.800000,10.0,4.366741
7,advanced high speed digital 6 feet hdmi 24k go...,home audio & theater,le,4.72,4.416667,60.0,4.344822
8,db9 male / db9 female null modem mini adapter,home audio & theater,bargain cable,4.85,4.875000,8.0,4.337128
9,barnes & noble industriell easel e-reader cover,portable audio & accessories,barnes & noble,0.00,4.588235,17.0,4.335231
10,premium 10 feet canon htc-100 upgrade replacem...,home audio & theater,a days tech,6.68,4.457143,35.0,4.334282


#### 5.3.2 Recommending Top 20 Products According to product_name

In [235]:
# E.g. cable
product_name = input("Enter the product name : ")

Enter the product name : cable


In [236]:
top_20_product_name = q_products[q_products['product_name'].str.contains(product_name.lower())]

top_20_product_name = top_20_product_name [columns].sort_values('score', ascending = False).reset_index()
top_20_product_name.index = top_20_product_name.index + 1

top_20_product_name.head(20)

,index,product_name,main_category,brand_or_author,price,rating_average,rating_count,score
1,1872,premium canon mini hdmi cable 10 feet,all electronics,lasus,0.00,4.442529,174.0,4.417338
2,865,bargain cable db9 female / db9 female null mod...,computers,stsi,5.05,4.800000,10.0,4.366741
3,760,advanced high speed digital 6 feet hdmi 24k go...,home audio & theater,le,4.72,4.416667,60.0,4.344822
4,1916,premium 10 feet canon htc-100 upgrade replacem...,home audio & theater,a days tech,6.68,4.457143,35.0,4.334282
5,1837,premium grade (6 foot) hdmi to mini hdmi cable...,all electronics,a days tech,0.00,4.615385,13.0,4.288154
6,2054,premium 50 foot high speed hdmi cable for your...,all electronics,a days tech,11.99,4.360000,25.0,4.193621
7,761,mobile excess advanced high speed digital 10 f...,all electronics,mobile excess,9.62,5.000000,5.0,4.172359
8,882,monster equivalent hdmi 1000hd ultra-high spee...,all electronics,monster,6.00,4.625000,8.0,4.114906
9,1868,premium 6 ft canon htc100 upgrade replacement ...,home audio & theater,a days tech,4.28,4.555556,9.0,4.103415
10,37,clikr-5 time warner cable remote control ur5u-...,all electronics,urc,0.00,4.500000,6.0,3.862022


#### 5.3.3 Recommending Top 20 Products According to main_category

In [237]:
# e.g computer
main_category = input("Enter the main category : ")

Enter the main category : computer


In [238]:
top_20_main_category = q_products[q_products['main_category'].str.contains(main_category.lower())]

top_20_main_category = top_20_main_category [columns].sort_values('score', ascending = False).reset_index()
top_20_main_category.index = top_20_main_category.index + 1

top_20_main_category.head(20)

,index,product_name,main_category,brand_or_author,price,rating_average,rating_count,score
1,1996,professional kingston microsdhc 4gb card for g...,computers,kingston,0.00,4.645161,62.0,4.571971
2,963,"perfect pixel 30"" yamakasi 301 sparta 2560x160...",computers,yamakasi,0.00,4.724138,29.0,4.567805
3,865,bargain cable db9 female / db9 female null mod...,computers,stsi,5.05,4.800000,10.0,4.366741
4,1621,professional ultra sandisk microsdxc 16gb (16 ...,computers,sandisk,9.50,4.451613,31.0,4.313567
5,1726,kingston professional microsdhc 8gb (8 gigabyt...,computers,kingston,9.37,4.777778,9.0,4.303415
6,1618,professional kingston microsdhc 8gb (8 gigabyt...,computers,kingston,7.90,5.000000,6.0,4.290593
7,848,8gb silver hello kitty crystal with necklace u...,computers,hello kitty,0.00,4.529412,17.0,4.279675
8,470,barnes & noble nook hd+ tablet 32gb slate (bnt...,computers,barnes & noble,89.55,4.319149,47.0,4.229878
9,1986,professional kingston microsdhc 4gb (4 gigabyt...,computers,kingston,0.00,4.750000,8.0,4.226017
10,1557,professional kingston microsdhc 32gb (32 gigab...,computers,kingston,9.50,4.833333,6.0,4.147736


#### 5.3.4 Recommending Top 20 Products According to brand_or_author

In [243]:
# e.g. barnes
brand_or_author = input("Enter the brand or author : ")

Enter the brand or author : barnes


In [244]:
top_20_brand_or_author = q_products[q_products['brand_or_author'].str.contains(brand_or_author.lower())]

top_20_brand_or_author = top_20_brand_or_author [columns].sort_values('score', ascending = False).reset_index()
top_20_brand_or_author.index = top_20_brand_or_author.index + 1

top_20_brand_or_author.head(20)

,index,product_name,main_category,brand_or_author,price,rating_average,rating_count,score
1,614,barnes noble 5010490303 lautner e-reader cover,portable audio & accessories,barnes & noble,22.49,4.809524,21.0,4.592462
2,634,barnes & noble industriell easel e-reader cover,portable audio & accessories,barnes & noble,0.00,4.588235,17.0,4.335231
3,622,barnes & noble emerson quote e-reader cover,portable audio & accessories,barnes & noble,29.99,4.777778,9.0,4.303415
4,61,barnes & noble dessin leather cover for nook c...,portable audio & accessories,barnes & noble,0.00,4.533333,15.0,4.252135
5,470,barnes & noble nook hd+ tablet 32gb slate (bnt...,computers,barnes & noble,89.55,4.319149,47.0,4.229878
6,72,barnes & noble ov/hb universal power kit for n...,computers,barnes & noble,0.00,4.384615,13.0,4.073868
7,84,barnes & noble hdtv adapter kit for nook hd an...,home audio & theater,barnes & noble,1.46,4.357143,14.0,4.068944
8,63,barnes & noble power adapter for nook simple t...,all electronics,barnes & noble,0.00,4.666667,6.0,4.004879
9,612,barnes & noble bn-adp-h01 power kit,portable audio & accessories,barnes & noble,58.88,3.875000,32.0,3.758611
10,460,"nook hd+ 9"" 16gb wi-fi color tablet",computers,barnes & noble,0.00,3.846154,26.0,3.704969


#### 5.3.5 Recommending Top 20 Products According to price range

In [245]:
# e.g. 10, 100
min_price = int(input("Enter the minimum price : "))
max_price = int(input("Enter the maximum price : "))

Enter the minimum price : 10
Enter the maximum price : 100


In [246]:
top_20_within_price_range = q_products[(q_products['price'] >= min_price) & (q_products['price'] <= max_price)]

top_20_within_price_range = top_20_within_price_range [columns].sort_values('score', ascending = False).reset_index()
top_20_within_price_range.index = top_20_within_price_range.index + 1

top_20_within_price_range.head(20)

,index,product_name,main_category,brand_or_author,price,rating_average,rating_count,score
1,614,barnes noble 5010490303 lautner e-reader cover,portable audio & accessories,barnes & noble,22.49,4.809524,21.0,4.592462
2,201,"videosecu 24"" long arm tv wall mount low profi...",all electronics,videosecu,34.99,4.470980,1051.0,4.466763
3,622,barnes & noble emerson quote e-reader cover,portable audio & accessories,barnes & noble,29.99,4.777778,9.0,4.303415
4,470,barnes & noble nook hd+ tablet 32gb slate (bnt...,computers,barnes & noble,89.55,4.319149,47.0,4.229878
5,2054,premium 50 foot high speed hdmi cable for your...,all electronics,a days tech,11.99,4.360000,25.0,4.193621
6,2035,kingston microsdhc 16gb,computers,kingston,33.97,4.571429,7.0,4.004269
7,992,external usb dvd/cd,computers,epartsdom,14.86,3.879397,199.0,3.860171
8,612,barnes & noble bn-adp-h01 power kit,portable audio & accessories,barnes & noble,58.88,3.875000,32.0,3.758611
9,446,"bluetooth headphones, bluetooth v4.1+edr wirel...",cell phones & accessories,cornmi,14.99,5.000000,3.0,3.758538
10,1732,professional ultra sandisk 32gb microsdhc veri...,computers,sandisk,14.27,5.000000,3.0,3.758538
